In [51]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
from pyspark.ml.feature import Tokenizer

In [2]:
from pyspark.ml.feature import CountVectorizer

In [3]:
from pyspark.ml.feature import MinHashLSH
from pyspark.ml.feature import BucketedRandomProjectionLSH

In [4]:
from pyspark.sql.functions import col, array, lit, length

In [89]:
!echo $JAVA_HOME

In [5]:
# from pyspark.context import SparkContext

In [6]:
from pyspark.sql import SparkSession

# logging.getLogger("py4j.java_gateway").setLevel(logging.ERROR)
spark = (
    SparkSession.builder
    .config("spark.driver.memory", "15g") 
    .getOrCreate()
)
# spark.sparkContext.setLogLevel(args.spark_log_level)

21/07/03 15:52:49 WARN Utils: Your hostname, wsy resolves to a loopback address: 127.0.1.1; using 10.1.0.140 instead (on interface enp34s0)
21/07/03 15:52:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/07/03 15:52:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [7]:
sc = spark.sparkContext

In [8]:
file="c4-nl.tfrecord-00000-of-01024.json.gz"

In [9]:
path="/media/yeb/NLP datasets/mc4/c4/multilingual"

In [62]:
df = spark.read.json(f"{path}/{file}")

In [63]:
df.show()

+--------------------+--------------------+--------------------+
|                text|           timestamp|                 url|
+--------------------+--------------------+--------------------+
|Vijf gouden tips ...|2019-02-22T15:37:25Z|https://ondernemi...|
|CuracaoNieuws | G...|2020-01-29T03:25:31Z|https://www.knips...|
|Nationaal Zwemcen...|2017-07-26T03:02:32Z|http://likealyzer...|
|Interim-managemen...|2020-04-01T21:11:42Z|http://www.interi...|
|Vastgoedbeheerder...|2019-10-14T10:47:53Z|https://vastgoedb...|
|dag 11 | WorldSup...|2020-08-15T01:30:29Z|https://www.world...|
|@Pierre Le Blanc#...|2020-07-02T22:18:51Z|http://nakoersen....|
|Henk Wieland (Ond...|2019-02-21T06:37:48Z|https://lentseluc...|
|Gamed.nl - Korte ...|2020-08-11T00:20:12Z|https://www.gamed...|
|GR - CORBION - 6/...|2016-10-28T08:22:03Z|http://www.belgie...|
|Informatiebijeenk...|2018-08-18T09:01:23Z|https://oncologie...|
|Onderooglidcorrec...|2019-07-22T18:10:54Z|https://www.dokte...|
|Kwark exotic – Pu...|202

In [64]:
df.count()

93956

In [65]:
df = df.limit(30000)

In [66]:
df = df.filter("text is not NULL").where(length(col("text"))>500)
df.count()

25792

In [67]:
tokenizer = Tokenizer(inputCol="text", outputCol="words")

In [68]:
words = tokenizer.transform(df)

In [69]:
words.show()

+--------------------+--------------------+--------------------+--------------------+
|                text|           timestamp|                 url|               words|
+--------------------+--------------------+--------------------+--------------------+
|Vijf gouden tips ...|2019-02-22T15:37:25Z|https://ondernemi...|[vijf, gouden, ti...|
|CuracaoNieuws | G...|2020-01-29T03:25:31Z|https://www.knips...|[curacaonieuws, |...|
|Nationaal Zwemcen...|2017-07-26T03:02:32Z|http://likealyzer...|[nationaal, zwemc...|
|Interim-managemen...|2020-04-01T21:11:42Z|http://www.interi...|[interim-manageme...|
|Vastgoedbeheerder...|2019-10-14T10:47:53Z|https://vastgoedb...|[vastgoedbeheerde...|
|dag 11 | WorldSup...|2020-08-15T01:30:29Z|https://www.world...|[dag, 11, |, worl...|
|Henk Wieland (Ond...|2019-02-21T06:37:48Z|https://lentseluc...|[henk, wieland, (...|
|Gamed.nl - Korte ...|2020-08-11T00:20:12Z|https://www.gamed...|[gamed.nl, -, kor...|
|Informatiebijeenk...|2018-08-18T09:01:23Z|https://onc

In [70]:
# mindf = minimum number of documents for a term to count
cv = CountVectorizer(inputCol="words", outputCol="features", vocabSize=10000, minDF=1)

In [71]:
model = cv.fit(words)

In [72]:
result = model.transform(words)

In [73]:
result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|           timestamp|                 url|               words|            features|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|Vijf gouden tips ...|2019-02-22T15:37:25Z|https://ondernemi...|[vijf, gouden, ti...|(10000,[0,1,2,3,4...|
|CuracaoNieuws | G...|2020-01-29T03:25:31Z|https://www.knips...|[curacaonieuws, |...|(10000,[0,1,2,3,4...|
|Nationaal Zwemcen...|2017-07-26T03:02:32Z|http://likealyzer...|[nationaal, zwemc...|(10000,[0,1,2,3,4...|
|Interim-managemen...|2020-04-01T21:11:42Z|http://www.interi...|[interim-manageme...|(10000,[0,1,2,3,4...|
|Vastgoedbeheerder...|2019-10-14T10:47:53Z|https://vastgoedb...|[vastgoedbeheerde...|(10000,[0,1,2,3,4...|
|dag 11 | WorldSup...|2020-08-15T01:30:29Z|https://www.world...|[dag, 11, |, worl...|(10000,[0,2,3,4,6...|
|Henk Wieland (Ond...|2019-02-21T06:3

In [74]:
# mh = MinHashLSH(inputCol="features", outputCol="hashes", numHashTables=1)
# model = mh.fit(result)

In [75]:
brp = BucketedRandomProjectionLSH(inputCol="features", outputCol="hashes", bucketLength=2.0,
                                  numHashTables=2)
model = brp.fit(result)

In [76]:
hashes = model.transform(result).repartition(100)

In [77]:
hashes.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+
|                text|           timestamp|                 url|               words|            features|          hashes|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+
|Nieuwe versie van...|2013-05-22T23:32:42Z|http://www.stambo...|[nieuwe, versie, ...|(10000,[0,1,2,3,4...| [[-1.0], [0.0]]|
|'); capa.innerHTM...|2020-04-06T11:46:00Z|https://www.camio...|[');, capa.innerh...|(10000,[187,245,3...| [[0.0], [-1.0]]|
|TIBCO heeft plann...|2020-07-02T14:42:25Z|https://biplatfor...|[tibco, heeft, pl...|(10000,[1,3,5,8,1...|  [[0.0], [0.0]]|
|Willibrord van Be...|2020-05-28T18:31:36Z|https://www.limbu...|[willibrord, van,...|(10000,[0,1,2,3,4...| [[0.0], [-1.0]]|
|Huis > Product Li...|2019-09-19T22:24:11Z|http://nl.sinowzh...|[huis, >, product...|(10000,[0,1,2,3,5...|[[-1.0], [-1.0]]|
|CUYLAER

In [78]:
from pyspark.ml.linalg import Vectors

In [79]:
#filter(hashes.hashes != array(array(lit(0))))
test = hashes.withColumn("this", hashes["hashes"].getItem(0)).select("this")
test.printSchema()
test.show(1)

root
 |-- this: vector (nullable = true)



+-----+
| this|
+-----+
|[1.0]|
+-----+
only showing top 1 row



In [80]:
# duplicates = model.approxSimilarityJoin(hashes, hashes, 0.01, distCol="JaccardDistance")\
#     .select(col("datasetA.url").alias("urlA"),
#             col("datasetB.url").alias("urlB"),
#             col("JaccardDistance"))\
#     .filter("urlA < urlB")

In [81]:
duplicates = model.approxSimilarityJoin(hashes, hashes, 0.1, distCol="EuclideanDistance")\
    .select(col("datasetA.url").alias("urlA"),
            col("datasetB.url").alias("urlB"),
            col("EuclideanDistance"))\
    .filter("urlA < urlB")

In [82]:
duplicates = duplicates.cache()

In [88]:
duplicates.count()

46

In [83]:
duplicates.show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------+
|urlA                                                                                                                                                                                                                                                                                               |urlB                                                                                                   

In [87]:
duplicates.show(100, truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------+
|urlA                                                                              

In [85]:
df.filter(col("url") == "http://www.jetcost.nl/autoverhuur-Bandar+Abbas,Iran.html").select("text").take(1)

[Row(text='Auto huren Bandar Abbas, BND goedkoop - Vergelijker van wagenverhuring Bandar Abbas (Iran) Jetcost.nl\nAutoverhuring Bandar Abbas\nVind uw goedkoop autoverhuur Bandar Abbas, Jetcost bergelijkt ogenblikkelijk duizende autoverhuurders Iran: Economisch, Compact, Wegverkeer, Mini, Break, Minibus, Comfort, Luxe. De vergelijker, Jetcost, is gespecialiseerd in de huur van wagens Bandar Abbas. Vind ook uw hotel Bandar Abbas en uw vlucht Bandar Abbas.\nAuto huren Bandar Abbas: Vergelijk uw autohuur Bandar Abbas om de voordeligste prijs te ontdekken. Alle promoties en disounts voor de autoverhuring Bandar Abbas bevinden zich op de Jetcost vergelijker. Voor uw huur bij uw aankomst op de luchthaven, in een station of een stadscentrum, gebruik onze vergelijker voor autoverhuur Bandar Abbas om minder te betalen.Vergelijk alle categorieën huurwagens (economische, standaard, polyvalent of bedrijfsvoertuig) met automatische of handbediende versnellingsbak en met of zonder air conditioning en

In [57]:
df.filter(col("url") == "http://www.jetcost.nl/autoverhuur-Dalyan,Turkije.html").select("text").take(1)

[Row(text='Auto huren Dalyan, goedkoop - Vergelijker van wagenverhuring Dalyan (Turkije) Jetcost.nl\nAutoverhuring Dalyan\nVind uw goedkoop autoverhuur Dalyan, Jetcost bergelijkt ogenblikkelijk duizende autoverhuurders Turkije: Economisch, Compact, Wegverkeer, Mini, Break, Minibus, Comfort, Luxe. De vergelijker, Jetcost, is gespecialiseerd in de huur van wagens Dalyan. Vind ook uw hotel Dalyan en uw vlucht Dalian.\nAuto huren Dalyan: Vergelijk uw autohuur Dalyan om de voordeligste prijs te ontdekken. Alle promoties en disounts voor de autoverhuring Dalyan bevinden zich op de Jetcost vergelijker. Voor uw huur bij uw aankomst op de luchthaven, in een station of een stadscentrum, gebruik onze vergelijker voor autoverhuur Dalyan om minder te betalen.Vergelijk alle categorieën huurwagens (economische, standaard, polyvalent of bedrijfsvoertuig) met automatische of handbediende versnellingsbak en met of zonder air conditioning en vind uw autoverhuring Dalyan tegen lage prijs. Alle merken, mod

In [61]:
df.filter(col("url") == "https://www.kamagra-100mg.nl/kamagra-overasselt/").select("text").take(1)

[Row(text='Kamagra Overasselt online kopen\n| kamagea | Kamagra Overasselt\nKamagra Overasselt\nZoekt u Kamagra Overasselt? Dan bent u bij kamagrakopen.nl aan het juiste adres. Kamagra kopen is een erectiemiddelen pharmacy die zich heeft gespecialiseerd in met name Kamagra maar ook een wijd assortiment aan andere erectiemiddelen levert. Wilt u snel de voor u juiste erectiepil proberen? Dan raden wij u aan om via deze link ons assortiment te bekijken. Wij bieden onze Kamagra in Den Haag aan en ook uiteraard in alle grote steden van Overasselt bieden wij onze erectiemiddelen aan.\nDe voordelen van Kamagra Overasselt:\nKamagra Overasselt Service\nKamagra Overasselt staat bekend om de snelle en sublieme service. Alle erectiemiddelen zijn zorgvuldig door ons geselecteerd om uw sexleven een optimale update te geven.. Al onze erectiemiddelen worden geselecteerd op diverse punten zoals werkzaamheid, geringe bijwerkingen en effectiviteit. Al onze inkopen controleren wij zorgvuldig voor u voor z

In [58]:
df.filter(col("url") == "https://www.kamagra-100mg.nl/kamagra-heumen/").select("text").take(1)

[Row(text='Kamagra Heumen online kopen\n| kamagea | Kamagra Heumen\nKamagra Heumen\nZoekt u Kamagra Heumen? Dan bent u bij kamagrakopen.nl aan het juiste adres. Kamagra kopen is een erectiemiddelen pharmacy die zich heeft gespecialiseerd in met name Kamagra maar ook een wijd assortiment aan andere erectiemiddelen levert. Wilt u snel de voor u juiste erectiepil proberen? Dan raden wij u aan om via deze link ons assortiment te bekijken. Wij bieden onze Kamagra in Den Haag aan en ook uiteraard in alle grote steden van Heumen bieden wij onze erectiemiddelen aan.\nDe voordelen van Kamagra Heumen:\nKamagra Heumen Service\nKamagra Heumen staat bekend om de snelle en sublieme service. Alle erectiemiddelen zijn zorgvuldig door ons geselecteerd om uw sexleven een optimale update te geven.. Al onze erectiemiddelen worden geselecteerd op diverse punten zoals werkzaamheid, geringe bijwerkingen en effectiviteit. Al onze inkopen controleren wij zorgvuldig voor u voor ze verzonden worden. Wij nemen al

In [15]:
df.write.format("json").save("/tmp/test")

In [102]:
def poly(x):
    n_dup = 0.000000036600* x**2 + 0.001349576537 * x -13.155795353500
    print(f"n_dups = {n_dup}, percentage = {n_dup / x}")

In [103]:
poly(10000)

n_dups = 3.999970016499997, percentage = 0.0003999970016499997


In [104]:
poly(25000)

n_dups = 43.4586180715, percentage = 0.00173834472286


In [105]:
poly(80000)


n_dups = 329.05032760649993, percentage = 0.004113129095081249


In [106]:
num = 1000*80000
poly(num)

n_dups = 234347952.9671646, percentage = 2.929349412089558
